# The Scrape phase
The objective here is to scrape some text and other meta-data from popular math blogs

In [0]:
from selenium import webdriver # The workhorse of this operation
from time import sleep # so we don't scrape too fast and get kicked out
import pandas as pd # for capturing the data in a tabular form
import string # for cleaning strings
import re # regular expressions for cleaning up some strings

In [0]:
driver = webdriver.Firefox()
driver.get('https://blog.feedspot.com/math_blogs/')

In [0]:
def test_function():
    print("Test function nar works.")

In [0]:
test_function()

In [0]:
bname_selectors = driver.find_elements_by_class_name('tlink')
math_selectors = driver.find_elements_by_class_name('ext')
text_selectors = driver.find_elements_by_xpath('//p[contains(@class,"trow trow-wrap")]')

In [0]:
bnames = [n.text.replace('|','').replace('?',' ').replace('/',' ').replace('-','') for n in bname_selectors]
math_urls = [mu.get_attribute("href") for mu in math_selectors]
text_ext = [f.text for f in text_selectors]

In [0]:
remove_wierd_chars = lambda s: "".join(filter(lambda char: char in string.printable, s))
def extract_post_rate(s):
    post_ind = s.find(' / ')
    s1 = s[post_ind-8:post_ind+10]
    n_ind = s1.find('\n')
    if n_ind < 5:
        s1 = s1[n_ind+1:]
        n_ind = s1.find('\n')
        s1 = s1[:n_ind]
    else:
        s1 = s1[:n_ind]
    return s1.strip()

In [0]:
clean_text_ext =[remove_wierd_chars(s) for s in text_ext]
post_rate_words = [extract_post_rate(cte) for cte in clean_text_ext ]

In [0]:
# Convert post rate into numbers
time_dict = {'day': 1,
             'week': 7,
             'mon': 30,
             'quar': 90,
             'year': 360} ;
def get_daily_post_rate(prw):
    if prw=='':
        daily_post_rate = 0
    else:
        digit = int(re.findall("\d",prw)[0])
        time_iterval = re.findall("day|week|mon|quar|year",prw)

        num_days = time_dict[time_iterval[0]]
        daily_post_rate = digit / num_days
    return daily_post_rate

In [0]:
daily_post_rates = [round(get_daily_post_rate(prw),ndigits=4) for prw in post_rate_words]

In [0]:
# store the scraped data
data_dict = {'blog_name' : bnames,
             'blog_url'  : math_urls,
             'post_rate_words' : post_rate_words,
             'daily_post_rate': daily_post_rates}
df = pd.DataFrame(data_dict)
df.to_excel('scrape_text\math_blogs.xlsx')

In [0]:
# store the scraped summary text
for t in range(len(bnames)):
    file2write=open('scrape_text\\'+ bnames[t]+"_desc.txt",'w')
    file2write.write(clean_text_ext[t])
    file2write.close() 

In [0]:
df = pd.read_excel('scrape_text\math_blogs.xlsx') # loading data that you have stored

## Some preliminary observations/visualizations.

In [0]:
import matplotlib.pyplot as plt

In [0]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax.set_title("Histogram of post rates");
ax.set_xlabel("Daily post rate");
ax.set_ylabel("Count");
ax.hist(df['daily_post_rate'],label="A",bins=15,histtype="step")

In [0]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax.set_title("A plot of sorted (asc.) daily post rate")
ax.scatter([i for i in range(75)],sorted(df['daily_post_rate']))
ax.set_xticks([])
# The flat streatches are due to the nature of our original data, e.g. posts / month etc.
# We can clearly see a powerlaw distribution as the most frequent posters distinguish them selves from the herd.

In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [0]:
engstopwords = nltk.corpus.stopwords.words('english') 
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
words = word_tokenize(' '.join(bnames))
lemmatized = [wordnet_lemmatizer.lemmatize(t.lower()) for t in words if t.lower() not in engstopwords]
counter = Counter(lemmatized)
most_common_words = counter.most_common(10)
w, c = zip(*most_common_words)

In [0]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax.set_title("Most common words in titles (by occurence)");
ax.bar(w, c)
plt.xticks(rotation=90);

# Scrape and Clean:
Since my spidermancy is lacking I have to do it somewhat manually. As such I scrape only the first 10 on the list:

In [0]:
import os
from collections import OrderedDict
def save_post_links(post_links_list, name):
    ds = pd.Series(post_links_list)
    filename = 'scrape_text\{}_post_links.xlsx'.format(name)
    ds.to_excel(filename)

In [0]:
url = df['blog_url'][0]
name = df['blog_name'][0].replace(' ','_')

## Scraping 'Stephen Wolfram Blog' 

In [0]:
driver = webdriver.Firefox()
driver.get(df['blog_url'][0])
post_links = []
for i in range(10):
    post_links_selectors = driver.find_elements_by_class_name("more-link")
    post_links.append([pls.get_attribute("href")  for pls in post_links_selectors])
    try:
        driver.find_element_by_id("__cookie-consent-button").click()
    except:
        0
    driver.find_element_by_xpath('//a[contains(@class,"page-numbers next")]').click()
    sleep(4)

In [0]:
post_links_list = [p for pl in post_links for p in pl]
save_post_links(post_links_list, name):
os.mkdir(name)
driver = webdriver.Firefox()

In [0]:
# look up the post page and scrape the content
for pl in post_links_list[2:]:
    try:
        driver.get(pl)
        sleep(4)
        post_content = driver.find_element_by_xpath('//div[@id="content"]').text
        post_title = pl.split('/')[-2]

        file2write=open(name+'\\'+post_title+".txt",'w')
        file2write.write(remove_wierd_chars(post_content))
        file2write.close() 
    except:
        print(pl)

## Scrape Math_Only_Math
This blog is more a collection of worksheets for elementary (like elementary school) mathematics exercises. As such there is not much in the line of words to scrape.

## Math_with_Bad_Drawings
This site use an "infite" scroll bar.

In [0]:
url = df['blog_url'][2]
name = df['blog_name'][2].replace(' ','_')
driver = webdriver.Firefox()
driver.get(url)
sleep(2)

In [0]:
post_titles  = []
post_links = []
for i in range(20):
#     post_title_selectors= driver.find_elements_by_xpath('//header[@class="entry-header"]')
    post_links_selectors= driver.find_elements_by_xpath('//a[@rel="bookmark"]')
    
    post_links_temp = [pls.get_attribute("href")  for pls in post_links_selectors]
    post_links.extend(post_links_temp)
#     post_titles.extend([pls.text for pls in post_title_selectors])
    sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [0]:
post_links = list(OrderedDict.fromkeys(post_links))
save_post_links(post_links, name)
os.mkdir(name)

In [0]:
for pl in post_links[1:]:
    sleep(3)
    try:
        driver.get(pl)
    #     post_content = driver.find_element_by_xpath('//div[@class,"entry-content"]').text
        post_content = driver.find_element_by_class_name("entry-content").text
        post_title = pl.split('/')[-2]

        file2write=open(name+'\\'+post_title+".txt",'w')
        file2write.write(remove_wierd_chars(post_content))
        file2write.close() 
    except:
        print(pl)

## "What's_new"

In [0]:
url = df['blog_url'][3]
name = df['blog_name'][3].replace(' ','_')
driver = webdriver.Firefox()
driver.get(url)
sleep(2)

In [0]:
post_links = []
for i in range(10):
    post_header_selectors = driver.find_elements_by_xpath('//h2[@class="post-title"]//a')
    post_links.extend([phs.get_attribute('href') for phs in post_header_selectors])
    sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
    try:
        driver.find_element_by_id("infinite-handle").find_element_by_xpath('//button').click()
    except:
        0

In [0]:
post_links = list(OrderedDict.fromkeys(post_links))
save_post_links(post_links, name)
os.mkdir(name)

In [0]:
for pl in post_links[10:]:
    sleep(3)
    try:
        driver.get(pl)
        post_content = driver.find_element_by_class_name("post-content" ).text
        post_title = pl.split('/')[-2]

        file2write=open(name+'\\'+post_title+".txt",'w')
        file2write.write(remove_wierd_chars(post_content))
        file2write.close()
    except:
        print(pl)

# Text analysis

In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

import os

In [0]:
def read_post(file):
    with open(file_dir+file) as f:
        lines = f.readlines()
        return ' '.join(lines).replace('\n','')
def get_number_of_chars(files):
    return [len(read_post(f)) for f in files]  
def get_number_of_words(files):
    return [len([t for t in word_tokenize(read_post(f)) if (t.lower() not in engstopwords) and (len(t) > 1 )]) for f in files ]

In [0]:
file_dir = 'scrape_text\\Stephen_Wolfram_Blog\\'
files = os.listdir(file_dir);
number_of_chars = get_number_of_chars(files)
number_of_words = get_number_of_words(files)

In [0]:
def make_hist_post_len(char_lens,s):
    fig, ax = plt.subplots(nrows=1,ncols=1)
    ax.set_title("Histogram of post lengths ({})".format(s));
    ax.set_xlabel("Number of {} per post".format(s));
    ax.set_ylabel("Count");
    ax.hist(char_lens,label="A",bins=20,histtype="step")

In [0]:
make_hist_post_len(char_lens,'characters')

In [0]:
make_hist_post_len(number_of_words,'words')

In [0]:
from wordcloud import WordCloud

In [0]:
 with open('scrape_text//rand.txt','r',encoding='utf-8-sig') as f:
        lines = f.readlines()
        text_m = ' '.join(lines).replace('\n','')

In [0]:
words = word_tokenize(text_m)
lemmatized = [wordnet_lemmatizer.lemmatize(t.lower()) for t in words if (t.lower() not in engstopwords) and (len(t) > 1)]

In [0]:
fig, axes = plt.subplots(1, 1)

text = ' '.join(lemmatized)
x, y = np.ogrid[:300, :300]
mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)

wc = WordCloud(background_color="black",
               repeat=True,
               mask=mask)
wc.generate(text)

plt.axis("off")
plt.imshow(wc, interpolation="bilinear")
plt.show()
fig.savefig('word cloud.jpg', bbox_inches='tight', dpi=250)